In [1]:
import sys
assert sys.version_info >= (3, 5)
# Python ≥3.5 is required
import sklearn
assert sklearn.__version__ >= "0.20"
import numpy as np
import os
import tarfile
import urllib
import pandas as pd
import urllib.request
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve
from sklearn import preprocessing
from sklearn import metrics
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import json
from pathlib import Path

In [2]:
data = pd.read_csv("02-19-2023.csv")

In [3]:
data

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2023-02-20 04:21:05,33.939110,67.709953,209107,7896,NaN,NaN,Afghanistan,537.158776,3.776057
1,NaN,NaN,NaN,Albania,2023-02-20 04:21:05,41.153300,20.168300,334314,3596,NaN,NaN,Albania,11616.999097,1.075635
2,NaN,NaN,NaN,Algeria,2023-02-20 04:21:05,28.033900,1.659600,271425,6881,NaN,NaN,Algeria,618.970454,2.535139
3,NaN,NaN,NaN,Andorra,2023-02-20 04:21:05,42.506300,1.521800,47866,165,NaN,NaN,Andorra,61950.430337,0.344712
4,NaN,NaN,NaN,Angola,2023-02-20 04:21:05,-11.202700,17.873900,105184,1931,NaN,NaN,Angola,320.036336,1.835831
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4011,NaN,NaN,NaN,West Bank and Gaza,2023-02-20 04:21:05,31.952200,35.233200,703228,5708,NaN,NaN,West Bank and Gaza,13784.956961,0.811686
4012,NaN,NaN,NaN,Winter Olympics 2022,2023-02-20 04:21:05,39.904200,116.407400,535,0,NaN,NaN,Winter Olympics 2022,NaN,0.000000
4013,NaN,NaN,NaN,Yemen,2023-02-20 04:21:05,15.552727,48.516388,11945,2159,NaN,NaN,Yemen,40.048994,18.074508
4014,NaN,NaN,NaN,Zambia,2023-02-20 04:21:05,-13.133897,27.849332,342317,4051,NaN,NaN,Zambia,1862.042098,1.183406


In [4]:
geoJSONDict = {

  "type": "FeatureCollection",
  "crs": {
    "type": "name",
    "properties": {
      "name": "urn:ogc:def:crs:OGC:1.3:CRS84"
    }
  },
  "features": []
}


In [5]:
# incident rate = (Cases/population)*100,000
# population = (Cases*100,000)/incident rate
#data["Population"] = (data["Confirmed"] * 1000)/data["Incident_Rate"]
#data["Incident_Rate"] = (data["Deaths"] / data["Population"])* 1000
data["Case_Fatality_Ratio"] = (data["Case_Fatality_Ratio"]*100).round()
data = data[data['Confirmed'] >= data['Deaths']]
data = data.sort_values(by='Incident_Rate', ascending=False)
data.head(50)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
3495,48301.0,Loving,Texas,US,2023-02-20 04:21:05,31.849476,-103.581857,396,1,NaN,NaN,"Loving, Texas, US",234319.526627,25.0
755,2068.0,Denali,Alaska,US,2023-02-20 04:21:05,63.672640,-150.007611,1801,2,NaN,NaN,"Denali, Alaska, US",85884.597043,11.0
765,2180.0,Nome,Alaska,US,2023-02-20 04:21:05,64.903207,-164.035380,7547,7,NaN,NaN,"Nome, Alaska, US",75439.824070,9.0
767,2188.0,Northwest Arctic,Alaska,US,2023-02-20 04:21:05,67.049192,-159.750395,5747,22,NaN,NaN,"Northwest Arctic, Alaska, US",75410.051174,38.0
3468,48247.0,Jim Hogg,Texas,US,2023-02-20 04:21:05,27.044539,-98.696819,3792,23,NaN,NaN,"Jim Hogg, Texas, US",72923.076923,61.0
1106,13053.0,Chattahoochee,Georgia,US,2023-02-20 04:21:05,32.343412,-84.788092,7886,24,NaN,NaN,"Chattahoochee, Georgia, US",72302.191253,30.0
189,NaN,NaN,Faroe Islands,Denmark,2023-02-20 04:21:05,61.892600,-6.911800,34658,28,NaN,NaN,"Faroe Islands, Denmark",70926.020669,8.0
750,2050.0,Bethel,Alaska,US,2023-02-20 04:21:05,60.909805,-159.856183,12837,47,NaN,NaN,"Bethel, Alaska, US",69819.427826,37.0
606,NaN,NaN,NaN,San Marino,2023-02-20 04:21:05,43.942400,12.457800,23521,122,NaN,NaN,San Marino,69305.792916,52.0
17,NaN,NaN,NaN,Austria,2023-02-20 04:21:05,47.516200,14.550100,5863481,21820,NaN,NaN,Austria,65103.493072,37.0


In [6]:
data.loc[data['Country_Region']=='Chad']

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
94,NaN,NaN,NaN,Chad,2023-02-20 04:21:05,15.4542,18.7322,7661,194,NaN,NaN,Chad,46.639874,253.0


In [7]:
data["Deaths"].sum()

6854432

In [8]:
data["Case_Fatality_Ratio"].sum()

574322.0

In [9]:
data["Incident_Rate"].sum()

111222738.00460437

In [10]:
us_data = data.loc[data['Country_Region']=='US']
data = data[data['Country_Region']!='US']

In [11]:
us_data

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
3495,48301.0,Loving,Texas,US,2023-02-20 04:21:05,31.849476,-103.581857,396,1,NaN,NaN,"Loving, Texas, US",234319.526627,25.0
755,2068.0,Denali,Alaska,US,2023-02-20 04:21:05,63.672640,-150.007611,1801,2,NaN,NaN,"Denali, Alaska, US",85884.597043,11.0
765,2180.0,Nome,Alaska,US,2023-02-20 04:21:05,64.903207,-164.035380,7547,7,NaN,NaN,"Nome, Alaska, US",75439.824070,9.0
767,2188.0,Northwest Arctic,Alaska,US,2023-02-20 04:21:05,67.049192,-159.750395,5747,22,NaN,NaN,"Northwest Arctic, Alaska, US",75410.051174,38.0
3468,48247.0,Jim Hogg,Texas,US,2023-02-20 04:21:05,27.044539,-98.696819,3792,23,NaN,NaN,"Jim Hogg, Texas, US",72923.076923,61.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3753,90051.0,Unassigned,Virginia,US,2020-12-21 13:27:30,NaN,NaN,0,0,NaN,NaN,"Unassigned, Virginia, US",NaN,NaN
3798,90053.0,Unassigned,Washington,US,2023-02-20 04:21:05,NaN,NaN,3303,6,NaN,NaN,"Unassigned, Washington, US",NaN,18.0
3852,90054.0,Unassigned,West Virginia,US,2021-07-31 23:21:38,NaN,NaN,0,0,NaN,NaN,"Unassigned, West Virginia, US",NaN,NaN
3922,90055.0,Unassigned,Wisconsin,US,2023-02-20 04:21:05,NaN,NaN,0,0,NaN,NaN,"Unassigned, Wisconsin, US",NaN,NaN


In [12]:
country_wise_data = pd.DataFrame()

In [13]:
#data struct to store states
# for every state in us_data: 
# get average of Case_Fatality_Ratio, median of lat and long, country
# insert row into data

countries =  data.Country_Region.values.tolist()
countries = list(dict.fromkeys(countries))

for country in countries:
    tempDf = data.loc[data['Country_Region']==country]
    countryDf = tempDf.groupby('Country_Region', as_index=False).Deaths.sum()
    case_fatality = tempDf.groupby('Country_Region', as_index=False).Case_Fatality_Ratio.median().iloc[0]['Case_Fatality_Ratio']
    lat = tempDf.groupby('Country_Region', as_index=False).Lat.median().iloc[0]['Lat']
    long = tempDf.groupby('Country_Region', as_index=False).Long_.median().iloc[0]['Long_']
  #  country = "US"
    countryDf["Lat"] = lat
    countryDf["Long_"] = long
    countryDf["Case_Fatality_Ratio"] = case_fatality
   # stateDf["Country_Region"] = country
    countryDf
    country_wise_data = country_wise_data.append(countryDf, ignore_index=True)

C:\Users\Shawn\AppData\Local\Temp\ipykernel_6124\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  country_wise_data = country_wise_data.append(countryDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_6124\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  country_wise_data = country_wise_data.append(countryDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_6124\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  country_wise_data = country_wise_data.append(countryDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_6124\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future versi

C:\Users\Shawn\AppData\Local\Temp\ipykernel_6124\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  country_wise_data = country_wise_data.append(countryDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_6124\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  country_wise_data = country_wise_data.append(countryDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_6124\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  country_wise_data = country_wise_data.append(countryDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_6124\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future versi

C:\Users\Shawn\AppData\Local\Temp\ipykernel_6124\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  country_wise_data = country_wise_data.append(countryDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_6124\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  country_wise_data = country_wise_data.append(countryDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_6124\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  country_wise_data = country_wise_data.append(countryDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_6124\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future versi

C:\Users\Shawn\AppData\Local\Temp\ipykernel_6124\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  country_wise_data = country_wise_data.append(countryDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_6124\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  country_wise_data = country_wise_data.append(countryDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_6124\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  country_wise_data = country_wise_data.append(countryDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_6124\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future versi

C:\Users\Shawn\AppData\Local\Temp\ipykernel_6124\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  country_wise_data = country_wise_data.append(countryDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_6124\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  country_wise_data = country_wise_data.append(countryDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_6124\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  country_wise_data = country_wise_data.append(countryDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_6124\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future versi

C:\Users\Shawn\AppData\Local\Temp\ipykernel_6124\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  country_wise_data = country_wise_data.append(countryDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_6124\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  country_wise_data = country_wise_data.append(countryDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_6124\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  country_wise_data = country_wise_data.append(countryDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_6124\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future versi

In [14]:
country_wise_data_scaled = country_wise_data
country_wise_data.sort_values(by='Case_Fatality_Ratio', ascending=False).head(100)

,Country_Region,Deaths,Lat,Long_,Case_Fatality_Ratio
196,MS Zaandam,2,NaN,NaN,2222.0
192,Yemen,2159,15.552727,48.516388,1807.0
185,Sudan,5011,12.862800,30.217600,786.0
171,Syria,3164,34.802075,38.996815,551.0
181,Somalia,1361,5.152149,46.199616,498.0
...,...,...,...,...,...
70,India,530761,23.780432,78.942140,107.0
139,Venezuela,5852,6.423800,-66.589700,106.0
147,Equatorial Guinea,183,1.650800,10.267900,106.0
35,Chile,64085,-35.046900,-71.475800,104.0


In [15]:
country_wise_data.quantile(0.25)

Deaths                 380.0000
Lat                      3.9193
Long_                   -8.2245
Case_Fatality_Ratio     47.5000
Name: 0.25, dtype: float64

In [16]:
country_wise_data.median()

C:\Users\Shawn\AppData\Local\Temp\ipykernel_6124\2100298474.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  country_wise_data.median()


Deaths                 3164.0000
Lat                      17.0608
Long_                    20.9394
Case_Fatality_Ratio     103.0000
dtype: float64

In [17]:
country_wise_data.quantile(0.75).round()

Deaths                 18506.0
Lat                       39.0
Long_                     47.0
Case_Fatality_Ratio      187.0
Name: 0.75, dtype: float64

In [18]:
country_wise_data_scaled["Case_Fatality_Ratio"] = country_wise_data_scaled["Case_Fatality_Ratio"]/2.28
country_wise_data_scaled["Deaths"] = country_wise_data_scaled["Deaths"]/2.28
country_wise_data.sort_values(by='Deaths', ascending=False).tail(100)

,Country_Region,Deaths,Lat,Long_,Case_Fatality_Ratio
171,Syria,1387.719298,34.802075,38.996815,241.666667
186,Nigeria,1383.771930,9.082000,8.675300,51.754386
151,Cambodia,1340.350877,11.550000,104.916700,96.491228
131,Kyrgyzstan,1311.842105,41.204380,74.766098,63.596491
27,Estonia,1283.333333,58.595300,25.013600,21.052632
...,...,...,...,...,...
60,Tuvalu,0.000000,-7.109500,177.649300,0.000000
194,Antarctica,0.000000,-71.949900,23.347000,0.000000
128,Holy See,0.000000,41.902900,12.453400,0.000000
197,Summer Olympics 2020,0.000000,35.649100,139.773700,0.000000


In [19]:
country_wise_data_scaled.quantile(0.25)

Deaths                 166.666667
Lat                      3.919300
Long_                   -8.224500
Case_Fatality_Ratio     20.833333
Name: 0.25, dtype: float64

In [20]:
country_wise_data_scaled.median()

C:\Users\Shawn\AppData\Local\Temp\ipykernel_6124\2592085398.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  country_wise_data_scaled.median()


Deaths                 1387.719298
Lat                      17.060800
Long_                    20.939400
Case_Fatality_Ratio      45.175439
dtype: float64

In [21]:
country_wise_data_scaled.quantile(0.75).round()

Deaths                 8117.0
Lat                      39.0
Long_                    47.0
Case_Fatality_Ratio      82.0
Name: 0.75, dtype: float64

In [22]:
#data struct to store states
# for every state in us_data: 
# get average of Case_Fatality_Ratio, median of lat and long, country
# insert row into data

states =  us_data.Province_State.values.tolist()
states = list(dict.fromkeys(states))

for state in states:
    tempDf = us_data.loc[us_data['Province_State']==state]
    stateDf = tempDf.groupby('Province_State', as_index=False).Case_Fatality_Ratio.mean()
    lat = tempDf.groupby('Province_State', as_index=False).Lat.median().iloc[0]['Lat']
    long = tempDf.groupby('Province_State', as_index=False).Long_.median().iloc[0]['Long_']
  #  country = "US"
    stateDf["Lat"] = lat
    stateDf["Long_"] = long
   # stateDf["Country_Region"] = country
    stateDf
    data = data.append(stateDf, ignore_index=True)

C:\Users\Shawn\AppData\Local\Temp\ipykernel_6124\1471081376.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(stateDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_6124\1471081376.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(stateDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_6124\1471081376.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(stateDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_6124\1471081376.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(stateDf, ignore_index=True)
C:\U

C:\Users\Shawn\AppData\Local\Temp\ipykernel_6124\1471081376.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(stateDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_6124\1471081376.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(stateDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_6124\1471081376.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(stateDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_6124\1471081376.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(stateDf, ignore_index=True)
C:\U

In [23]:
clean_data_case_fatality = data.drop(['FIPS', 'Admin2','Province_State','Country_Region','Last_Update','Incident_Rate','Recovered','Active','Combined_Key','Deaths','Confirmed'], axis=1)
clean_data_case_fatality = clean_data_case_fatality.dropna()
clean_data_case_fatality.loc[~(clean_data_case_fatality==0.0).all(axis=1)]
clean_data_case_fatality = clean_data_case_fatality.sort_values(by='Case_Fatality_Ratio', ascending=False)
clean_data_case_fatality.head(50)

,Lat,Long_,Case_Fatality_Ratio
688,15.552727,48.516388,1807.0
605,19.566500,-101.706800,789.0
675,12.862800,30.217600,786.0
588,19.041400,-98.206300,764.0
599,19.173800,-96.134200,731.0
464,-6.353049,-79.824113,730.0
485,-5.133361,-80.335861,729.0
505,-9.621718,-73.444929,720.0
558,30.840600,-115.283800,719.0
392,-14.235097,-75.574821,700.0


In [24]:
clean_data_deaths = data.drop(['FIPS', 'Admin2','Province_State','Country_Region','Last_Update','Incident_Rate','Recovered','Active','Combined_Key','Case_Fatality_Ratio','Confirmed'], axis=1)
clean_data_deaths = clean_data_deaths.dropna()
clean_data_deaths.loc[~(clean_data_deaths==0.0).all(axis=1)]
clean_data_deaths = clean_data_deaths.sort_values(by='Deaths', ascending=False)
clean_data_deaths.head(50)

,Lat,Long_,Deaths
118,52.355500,-1.174300,184819.0
373,-23.550500,-46.633300,178840.0
12,46.227600,2.213700,161090.0
603,-0.789300,113.921300,160886.0
525,19.449759,76.108221,148421.0
479,32.427908,53.688046,144804.0
262,-38.416100,-63.616700,130448.0
321,51.919400,19.145100,118826.0
521,-30.559500,22.937500,102595.0
284,38.963700,35.243300,101492.0


In [25]:
#The mistake made here is Im rendering EACH AND EVERY covid case 
#Instead of that, create an object with long, lat and total case count.
#Hence we should only have approx 4000 json objects instead of 600 million+
#TODO: Figure out how mapbox is gonna render it



for i in range(len(clean_data_case_fatality)):
    
    longitude = clean_data_case_fatality.iloc[i]['Long_']
    latitude = clean_data_case_fatality.iloc[i]['Lat']
    data_points = int(clean_data_case_fatality.iloc[i]['Case_Fatality_Ratio'])
    
    
    for x in range(data_points):
        geometry = {
        "geometry": {
          "coordinates": [
            longitude,
            latitude
          ]
        }
      }
        geoJSONDict["features"].append(geometry)
    

json_object = json.dumps(geoJSONDict, separators=(',', ":"))
with open("../server/case_fatality_ratio.json", "w") as outfile:
    outfile.write(json_object)

    
geoJSONDict = {

  "type": "FeatureCollection",
  "crs": {
    "type": "name",
    "properties": {
      "name": "urn:ogc:def:crs:OGC:1.3:CRS84"
    }
  },
  "features": []
}
    
    
    

for i in range(len(clean_data_deaths)):
    
    longitude = clean_data_deaths.iloc[i]['Long_']
    latitude = clean_data_deaths.iloc[i]['Lat']
    data_points = int(clean_data_deaths.iloc[i]['Deaths'])
    
    
    for x in range(data_points):
        geometry = {
        "geometry": {
          "coordinates": [
            longitude,
            latitude
          ]
        }
      }
        geoJSONDict["features"].append(geometry)
    

json_object = json.dumps(geoJSONDict, separators=(',', ":"))
with open("../server/deaths.json", "w") as outfile:
    outfile.write(json_object)
    

## Creating the graphs

In [26]:
vaccination_data = pd.read_csv("vaccinations-by-manufacturer.csv")

In [27]:
vaccination_data

,location,date,vaccine,total_vaccinations
0,Argentina,2020-12-29,Oxford/AstraZeneca,1
1,Argentina,2020-12-29,Sinopharm/Beijing,1
2,Argentina,2020-12-29,Sputnik V,20491
3,Argentina,2020-12-30,Sputnik V,40592
4,Argentina,2020-12-31,Sputnik V,43398
...,...,...,...,...
57665,European Union,2023-03-21,Sanofi/GSK,4532
57666,European Union,2023-03-21,Sinopharm/Beijing,2319921
57667,European Union,2023-03-21,Sinovac,8891
57668,European Union,2023-03-21,Sputnik V,1845386


In [28]:
vaccination_data.dropna()

,location,date,vaccine,total_vaccinations
0,Argentina,2020-12-29,Oxford/AstraZeneca,1
1,Argentina,2020-12-29,Sinopharm/Beijing,1
2,Argentina,2020-12-29,Sputnik V,20491
3,Argentina,2020-12-30,Sputnik V,40592
4,Argentina,2020-12-31,Sputnik V,43398
...,...,...,...,...
57665,European Union,2023-03-21,Sanofi/GSK,4532
57666,European Union,2023-03-21,Sinopharm/Beijing,2319921
57667,European Union,2023-03-21,Sinovac,8891
57668,European Union,2023-03-21,Sputnik V,1845386


In [29]:
vaccination_data.drop(['location','date'],axis=1)

,vaccine,total_vaccinations
0,Oxford/AstraZeneca,1
1,Sinopharm/Beijing,1
2,Sputnik V,20491
3,Sputnik V,40592
4,Sputnik V,43398
...,...,...
57665,Sanofi/GSK,4532
57666,Sinopharm/Beijing,2319921
57667,Sinovac,8891
57668,Sputnik V,1845386


In [30]:
vaccines_list = vaccination_data.vaccine.unique().tolist()
vaccine_dict = {}
for i in range(len(vaccines_list)):
    vaccine_dict[vaccines_list[i]] = 0
vaccine_dict

{'Oxford/AstraZeneca': 0,
 'Sinopharm/Beijing': 0,
 'Sputnik V': 0,
 'Pfizer/BioNTech': 0,
 'Moderna': 0,
 'CanSino': 0,
 'Sputnik Light': 0,
 'Johnson&Johnson': 0,
 'Novavax': 0,
 'Sanofi/GSK': 0,
 'Valneva': 0,
 'Medicago': 0,
 'Sinovac': 0,
 'Covaxin': 0,
 'SKYCovione': 0}

In [31]:
vaccine_dict = {'Oxford/AstraZeneca': 0,
 'Sinopharm/Beijing': 0,
 'Pfizer/BioNTech': 0,
 'Moderna': 0,
 'Johnson&Johnson': 0,
 'Others':0}
vaccine_dict

{'Oxford/AstraZeneca': 0,
 'Sinopharm/Beijing': 0,
 'Pfizer/BioNTech': 0,
 'Moderna': 0,
 'Johnson&Johnson': 0,
 'Others': 0}

In [32]:
for i in range(len(vaccination_data)):
    currVaccine = vaccination_data.iloc[i]['vaccine']
    currCount = vaccination_data.iloc[i]['total_vaccinations']
    
    if currVaccine == 'Oxford/AstraZeneca' or currVaccine == 'Sinopharm/Beijing' or currVaccine == 'Pfizer/BioNTech' or currVaccine == 'Moderna' or currVaccine == 'Johnson&Johnson' :
        dictCount = vaccine_dict[currVaccine]
        vaccine_dict[currVaccine] = int(currCount + dictCount)
    else:
        othersCount = vaccine_dict['Others']
        vaccine_dict['Others'] = int(currCount + dictCount)

In [33]:
vaccine_dict

{'Oxford/AstraZeneca': 82093563042,
 'Sinopharm/Beijing': 31021031484,
 'Pfizer/BioNTech': 939904167926,
 'Moderna': 263357674637,
 'Johnson&Johnson': 26013600382,
 'Others': 31018721338}

In [34]:
json_object = json.dumps(vaccine_dict, separators=(',', ":"))
with open("../server/vaccines.json", "w") as outfile:
    outfile.write(json_object)